<html>
<head>
<style>
body {background-color: powderblue;}
h1   {color: blue}
<style>
</head>
<body>

<h1>Movies Filmed in Vermont</>

In [11]:
<h1 style="background-color:MediumSeaGreen;"><strong> Movies filmed in Vermont </strong></h1>

<h3>Interactive Map - Click on a marker to reveal which movie was filmed where</h3
    
<p> A list of the most popular movies to be filmed in Vermont, though most of them were not entirely shot in Vermont. Fun fact, Bernie Sanders had a cameo in Sweet Hearsts Dance<p>




![Vermont_state_flag.png](attachment:4cc620e0-6302-4696-a187-ad3389b085d8.png)

 State flag of Vermont

---

The libaries to be imported

from flask import Flask
import folium
from folium import plugins
from branca.element import Figure
import pandas as pd
import geopandas as gpd

Set the default size of our map

fig = Figure(width=550, height=550)

Read in the jsosn file that will outline the counties

coords = gpd.read_file("vermont-county-boundaries.geojson")
coords.head()

Read csv file containing data of the movies

movies = pd.read_csv('VTmoviesCoords.csv')
movies.head()

Drop anything we do not need

movies.drop(columns=['Other Info'],axis=1, inplace=True)
movies.head(2)

General info about each column in the dataset

movies.info()

Create the map and set a few parameters

map = folium.Map(location=[44.000000, -72.699997],
                 zoom_start=5, 
                 min_zoom=8, 
                 max_zoom=22,
                 tiles ="http://services.arcgisonline.com/arcgis/rest/services/NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}",
                 attr= "Sources: National Geographic, Esri, Garmin")

Iterate through the rows of the jsosn file and add the polygon outlines to our base map

for _, r in coords.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                          style_function=lambda x: {
                              'color': 'black',
                              'weight': 2.5,
                              'fillOpacity': 0
                          },name='County').add_to(map)
    folium.Popup(r['cntyname']).add_to(geo_j)
    geo_j.add_to(map)

A function with some html to define what are pop up markers will display

def popup_html(row):
    i = row
    lat = movies['Latitude'].iloc[i]
    lon = movies['Longitude'].iloc[i]
    year = movies['Year'].iloc[i]
    title = movies['Title'].iloc[i]
    town = movies['Town'].iloc[i]
    url = movies['URL'].iloc[i]
    star = movies['Starring'].iloc[i]

    left_col_color = "#85929E"
    right_col_color = "#f2f9ff"

    html = """
    <!DOCTYPE html>
    <html>
    <center><h4 style="margin-bottom:5"; width="200px">{}</h4>""".format(title) + """</center>
        
    <center><a href=\"""" + url + """\">Click here to view the movie's IMDb page</a></center>
    
    <center> <table style="height: 115px; width: 250px;">
    
    <tbody>
    <figure>
     <a title="Internet Movie Database, Public domain, via Wikimedia Commons" 
     href="https://commons.wikimedia.org/wiki/File:IMDB_Logo_2016.svg">
     <img width="100" alt="IMDB Logo 2016" 
     src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/IMDB_Logo_2016.svg/512px-IMDB_Logo_2016.svg.png">
     </a>
    </figure
    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Town filmed in </span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">"""+""+town + """</td>
    </tr>
    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Year of filming </span></td>
    <td style="width: 130px;background-color: """+ right_col_color +""";">{}</td>""".format(year) + """
    
    <tr>
   <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Starring </span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">"""+ star + """</td>

    </tbody>
    </table></center>
    </html>
    """
    return html

Go through the movie data and define color for markers depending on Year.
Set location of each marker based on coordinates of each film.

Add the function, icons and everything else to main basemap.

for i in range(0, len(movies)):
    year = movies['Year'].iloc[i]
    if year <= 1989:
        color = 'green'
    elif year <= 1999:
        color = 'orange'
    else:
        color = 'blue'
    html = popup_html(i)
    popup= folium.Popup(folium.Html(html, script=True), max_width=500)
    folium.Marker([movies['Latitude'].iloc[i], 
                           movies['Longitude'].iloc[i]],
                           popup=popup,
                           icon=folium.Icon(color=color, icon='fa-imdb', prefix='fa')).add_to(map)

Display the updated map

map
#map.save("Movies_In_Vermont.html")

In [12]:
map.save("Movies_In_Vermont.html")